In [1]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.svm import SVR
from math import sqrt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import math
from tensorflow.keras import optimizers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn import metrics
from  sklearn.datasets  import  make_hastie_10_2
from xgboost.sklearn import XGBClassifier


In [2]:
import torch

In [3]:
# 原始資料
'''
columns = ['Age', 'Workclass', 'fnlwgt', 'education'
    , 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
           'hours-per-week', 'native-country', 'salary']
'''
columns = ['Age','Workclass','fnlwgt','Education'
    ,'EdNum','MaritalStatus','Occupation','Relationship','Race','Sex','CapitalGain','CapitalLoss',
           'HoursPerWeek','Country','Income']


df_train_set = pd.read_csv('datasets/adult.data', names=columns, na_values=' ?')
df_test_set = pd.read_csv('datasets/adult.test', names=columns, skiprows=1, na_values=' ?')

print(df_train_set.shape)
print(df_test_set.shape)
df_test_set.head(30)
#print(df_train_set)
#print(df_test_set)

(32561, 15)
(16281, 15)


,Age,Workclass,fnlwgt,Education,EdNum,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HoursPerWeek,Country,Income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K.
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K.
6,29,NaN,227026,HS-grad,9,Never-married,NaN,Unmarried,Black,Male,0,0,40,United-States,<=50K.
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K.
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K.
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K.


In [4]:
df_train_set.drop('fnlwgt', axis=1, inplace=True)
df_test_set.drop('fnlwgt', axis=1, inplace=True)
df_train_set = df_train_set.dropna()
df_test_set = df_test_set.dropna()
print(df_train_set.shape)
print(df_test_set.shape)
df_test_set.head()
#print(adult_data)
#print(adult_test)

(30162, 14)
(15060, 14)


,Age,Workclass,Education,EdNum,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HoursPerWeek,Country,Income
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
5,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K.


In [5]:
df_train_set.drop(["Country", "Education"], axis=1, inplace=True) 
df_test_set.drop(["Country", "Education"], axis=1, inplace=True)

In [6]:
print(df_train_set.shape)
print(df_test_set.shape)

(30162, 12)
(15060, 12)


In [7]:
colnames = list(df_train_set.columns) 
colnames.remove('Age')
colnames.remove('EdNum')
colnames = ['AgeGroup', 'EduGroup']+colnames

labels = ["{0}-{1}".format(i, i+9) for i in range(0,100,10)]
df_train_set['AgeGroup'] = pd.cut(df_train_set.Age, range(0,101,10), right = False, labels = labels) 
df_test_set['AgeGroup'] = pd.cut(df_test_set.Age, range(0,101,10), right = False, labels = labels) 

labels = ["{0}-{1}".format(i,i+4) for i in range(0,20,5)]    
df_train_set['EduGroup'] = pd.cut(df_train_set.EdNum, range(0,21,5), right = False, labels = labels)   
df_test_set['EduGroup'] = pd.cut(df_test_set.EdNum, range(0,21,5), right = False, labels = labels)

df_train_set = df_train_set[colnames] 
df_test_set = df_test_set[colnames]
df_train_set.shape
df_train_set.head()

,AgeGroup,EduGroup,Workclass,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HoursPerWeek,Income
0,30-39,10-14,State-gov,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,<=50K
1,50-59,10-14,Self-emp-not-inc,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,<=50K
2,30-39,5-9,Private,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,<=50K
3,50-59,5-9,Private,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,<=50K
4,20-29,10-14,Private,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,<=50K


In [8]:
from sklearn_pandas import DataFrameMapper
mapper = DataFrameMapper([('AgeGroup', LabelEncoder()),('EduGroup', LabelEncoder()),
                          ('Workclass', LabelEncoder()),('MaritalStatus', LabelEncoder()),
                          ('Occupation', LabelEncoder()),('Relationship', LabelEncoder()),
                          ('Race', LabelEncoder()),('Sex', LabelEncoder()),
                          ('Income', LabelEncoder())], df_out=True, default=None)

cols = list(df_train_set.columns)
print(cols)
cols.remove('Income')
print(cols)
cols = cols[:-3]+['Income']+cols[-3:]  

df_train = mapper.fit_transform(df_train_set.copy())
df_train.columns = cols
df_test = mapper.fit_transform(df_test_set.copy())
df_test.columns = cols

train_minmax = preprocessing.MinMaxScaler(feature_range=(0, 1))
train_data_minmax = train_minmax.fit_transform(df_train[cols])
test_data_minmax = train_minmax.transform(df_test[cols])

df_train = pd.DataFrame(train_data_minmax,columns =cols)
df_test = pd.DataFrame(test_data_minmax,columns =cols)

df_train.head()
'''
df_train.loc[df_train.Income==-1,'Income'] = 0
df_train.loc[df_train.Income==1,'Income'] = 1

df_test.loc[df_test.Income==-1,'Income'] = 0
df_test.loc[df_test.Income==1,'Income'] = 1
'''

cols.remove('HoursPerWeek')

['AgeGroup', 'EduGroup', 'Workclass', 'MaritalStatus', 'Occupation', 'Relationship', 'Race', 'Sex', 'CapitalGain', 'CapitalLoss', 'HoursPerWeek', 'Income']
['AgeGroup', 'EduGroup', 'Workclass', 'MaritalStatus', 'Occupation', 'Relationship', 'Race', 'Sex', 'CapitalGain', 'CapitalLoss', 'HoursPerWeek']


In [9]:
df_train.head()

,AgeGroup,EduGroup,Workclass,MaritalStatus,Occupation,Relationship,Race,Sex,Income,CapitalGain,CapitalLoss,HoursPerWeek
0,0.250,0.333333,0.833333,0.666667,0.000000,0.2,1.0,1.0,0.0,0.02174,0.0,0.397959
1,0.500,0.333333,0.666667,0.333333,0.230769,0.0,1.0,1.0,0.0,0.00000,0.0,0.122449
2,0.250,1.000000,0.333333,0.000000,0.384615,0.2,1.0,1.0,0.0,0.00000,0.0,0.397959
3,0.500,1.000000,0.333333,0.333333,0.384615,0.0,0.5,1.0,0.0,0.00000,0.0,0.397959
4,0.125,0.333333,0.333333,0.333333,0.692308,1.0,0.5,0.0,0.0,0.00000,0.0,0.397959


In [10]:
x_train, y_train = df_train[cols].values, df_train['HoursPerWeek'].values
x_test, y_test = df_test[cols].values, df_test['HoursPerWeek'].values

x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2,random_state=42)
df_train.head(10)
print(df_train.shape)
print(df_test.shape)

(30162, 12)
(15060, 12)


In [11]:
import tensorflow.keras.backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [12]:
def mape_loss_func(labels,preds):
    mask=labels!=0
    return K.fabs((preds[mask]-labels[mask])/labels[mask]).mean()

In [13]:
from tensorflow.keras.callbacks import EarlyStopping

In [14]:
callbacks = [
            EarlyStopping(patience = 20,mode='max',monitor='val_loss')
            ]

In [15]:
model = keras.Sequential(name='model-1')
model.add(layers.Dense(64, activation='relu',input_shape=(11,)))
model.add(layers.Dense(32, activation='relu' ))
model.add(layers.Dense(32, activation='relu' ))
model.add(layers.Dropout(0.15))
model.add(layers.Dense(16, activation='relu' ))
model.add(layers.Dense(1))
model.summary()

model.compile(keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
             loss=keras.losses.MeanSquaredError())

hisroty=model.fit(x_train,y_train,
                 batch_size=128,
                 epochs=1000,
                 callbacks=callbacks,
                validation_data=(x_validation, y_validation)
                 )


Model: "model-1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                768       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 4,449
Trainable params: 4,449
Non-trainable params: 0
_________________________________________________________

In [20]:
y_pred=model.predict(x_test)
print(y_pred)
print(y_test)

[[0.36195424]
 [0.43299487]
 [0.4585183 ]
 ...
 [0.431776  ]
 [0.41159004]
 [0.4791477 ]]
[0.39795918 0.5        0.39795918 ... 0.5        0.39795918 0.60204082]


In [21]:
def mape_loss_func(labels,preds):
    mask=labels!=0
    return np.fabs((preds[mask]-labels[mask])/labels[mask]).mean()

In [23]:
print("RMSE : %.4g" % metrics.mean_squared_error(y_test, y_pred,squared=False))
print("MAPE : %.4g" % mape_loss_func(y_test,y_pred))

RMSE : 0.1095
MAPE : 0.3804
